# (Result) by (action) using (feature)
<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->

Introductory paragraph - for example:

This tutorial demonstrates how to work with [feature](link to feature doc). In this tutorial you perform the following tasks:

- Task 1
- Task 2
- Task 3
- etc

## Prerequisites

This tutorial works with Druid XX.0.0 or later.

#### Run with Docker

Launch this tutorial and all prerequisites using the `druid-jupyter` profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see the Learn Druid repository [readme](https://github.com/implydata/learn-druid).

## Initialization

The following cells set up the notebook and learning environment ready for use.

### Set up and connect to the learning environment

Run the next cell to set up the Druid Python client's connection to Apache Druid.

If successful, the Druid version number will be shown in the output.

In [ ]:
import druidapi
import os

if 'DRUID_HOST' not in os.environ.keys():
    druid_host=f"http://localhost:8888"
else:
    druid_host=f"http://{os.environ['DRUID_HOST']}:8888"
    
print(f"Opening a connection to {druid_host}.")
druid = druidapi.jupyter_client(druid_host)

display = druid.display
sql_client = druid.sql
status_client = druid.status
status_client.version

### Load example data

Run the following cell to create a table called `example-koalas-null`. Notice {the use of X as a timestamp | only required columns are ingested | WHERE / expressions / GROUP BY are front-loaded | partitions on X period and clusters by Y}.

When completed, you'll see a description of the final table.

> During ingestion, NULL values are recorded differently based on whether [SQL-compatible NULL handling](https://druid.apache.org/docs/latest/querying/sql-data-types#null-values) is on. Read more in the [handling null values](https://druid.apache.org/docs/latest/design/segments/#handling-null-values) documentation. As this notebook presumes the use of Druid 28.0.0 and above, which uses this mode by default, string columns that are truly NULL are stored as a NULL, rather than an empty string, while NULLs in numeric data will be stored as true NULL rather than a zero.

In [ ]:
sql='''
REPLACE INTO "example-koalas-null" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["https://static.imply.io/example-data/kttm-v2/kttm-v2-2019-08-25.json.gz"]}',
    '{"type":"json"}'
  )
) EXTEND ("timestamp" VARCHAR, "agent_category" VARCHAR, "agent_type" VARCHAR, "browser" VARCHAR, "browser_version" VARCHAR, "city" VARCHAR, "continent" VARCHAR, "country" VARCHAR, "version" VARCHAR, "event_type" VARCHAR, "event_subtype" VARCHAR, "loaded_image" VARCHAR, "adblock_list" VARCHAR, "forwarded_for" VARCHAR, "language" VARCHAR, "number" VARCHAR, "os" VARCHAR, "path" VARCHAR, "platform" VARCHAR, "referrer" VARCHAR, "referrer_host" VARCHAR, "region" VARCHAR, "remote_address" VARCHAR, "screen" VARCHAR, "session" VARCHAR, "session_length" BIGINT, "timezone" VARCHAR, "timezone_offset" VARCHAR, "window" VARCHAR))
SELECT
  TIME_PARSE("timestamp") AS "__time",
  "referrer",
  CASE WHEN "referrer" = 'Direct' THEN NULL
    ELSE "referrer" END AS "referrer-null",
  "loaded_image",
  "event_type",
  "event_subtype",
  CASE WHEN ("event_type" = 'PercentClear' AND "event_subtype" = '') THEN 0 ELSE "event_subtype" END AS "event_subtype-zeroed",
  "session",
  "session_length",
  CASE WHEN ("timezone" = 'EDT') THEN "session_length" ELSE NULL END AS "session_length-EDTonly",
  "platform",
  "agent_category",
  "continent",
  "language",
  "timezone",
  CASE WHEN "timezone" = 'N/A' THEN NULL
    ELSE "timezone"
    END AS "timezone-null"
FROM "ext"
PARTITIONED BY DAY
'''

display.run_task(sql)
sql_client.wait_until_ready('example-koalas-null')
display.table('example-koalas-null')

## A note on NULL handling

There are two modes for [NULL-handling](https://druid.apache.org/docs/latest/querying/sql-data-types#null-values) in Apache Druid. Define what mode to use by setting the `druid.generic.useDefaultValueForNull` runtime property.

This notebook presumes that this setting is `false`, which is the default since Apache Druid 28.0.0.

## Testing for NULL

During ingestion, a new dimension, `referrer-null` was created. A CASE statement was applied to the incoming `referrer` so that anything recorded as "Direct" is instead stored as a NULL.

Run the following cell to see how the original data (`referrer`) has been mapped to a NULL in `referrer-null`.

In [ ]:
sql='''
SELECT
  COUNT(*) FILTER (WHERE "referrer" = 'Direct') AS "referrer",
  COUNT(*) FILTER (WHERE "referrer-null" IS NULL) AS "referrer-null"
FROM "example-koalas-null"
'''

display.sql(sql)

In [ ]:
A second CASE statement during ingestion 

In [ ]:
sql='''
SELECT
  COUNT(*) FILTER (WHERE "event_subtype" = '') AS "event_subtype",
  COUNT(*) FILTER (WHERE "event_subtype-null" IS NULL) AS "event_subtype-null"
FROM "example-koalas-null"
'''

display.sql(sql)

In the source data, `PercentClear`-type events are recorded as people interact with an image on the site. `event_subtype` then records a percentage clear, from zero through to 100 percent.

However, in the source data, zero values are empty, instead of containing a true zero.

The following CASE statement therefore checks for empty values and replaces it with a true zero at ingestion time. Notice how Druid is careful not to intepret an empty string in the source data as equivalent to NULL.

```sql
CASE WHEN ("event_type" = 'PercentClear' AND "event_subtype" = '') THEN 0 ELSE "event_subtype" END AS "event_subtype-zeroed"
```

Run the following cell to see how this shows up in the data.

In [ ]:
sql='''
SELECT
  CONCAT("event_subtype-zeroed",'%') AS "Percentage Cleared",
  COUNT(*) AS "events"
FROM "example-koalas-null"
WHERE "event_type" = 'PercentClear'
GROUP BY "event_subtype-zeroed"
ORDER BY CAST("event_subtype-zeroed" AS DOUBLE)
'''

display.sql(sql)

## Conditions

Use NVL and COALESCE to return another value when the expression is NULL.

> `COALESCE(col1, col2)` always returns `col1` when SQL-compatible NULL-handling is off since empty strings and NULL are considered equivallent. Read more in the [NULL-handling](https://druid.apache.org/docs/latest/querying/sql-data-types/#null-values) documentation.

A CASE statement in the ingestion SQL outputs a field `example-koalas-null`, translating any incoming value of "N/A" to a true NULL.

Run the following cell to see examples of these functions being used.

In [ ]:
sql='''
SELECT COALESCE("timezone-null",'UTC') AS "timezone",
COUNT(*) AS "events"
FROM "example-koalas-null"
GROUP BY 1
HAVING COUNT(*) > 5000
'''

display.sql(sql)

Notice that in the ingestion SQL statement, `session_length-null` is output as a NULL when the `session_length` is zero.

In [ ]:
sql='''
SELECT COUNT(*)
FROM "example-koalas-null"
WHERE "session_length-null" IS NULL
'''

display.sql(sql)

## Boolean operators

Use the `||` and `&&` operators in a query as a shorthand for OR and AND respectively.

> Apache Druid uses SQL-compatible mode for boolean operators. Read more about how to turn [strict boolean behaviour](https://druid.apache.org/docs/latest/misc/math-expr/#logical-operator-modes) on or off in coniguration through `druid.expressions.useStrictBooleans`.

* The first two results are considered TRUE.
* The remaining calculations return NULL.

In [ ]:

SELECT
true || null,
null || true,
false || null,
null || false,
null || null
FROM x


Run the next cell for examples of the AND operator.

* The first calculation returns TRUE as both functions return TRUE.
* The next two calculations return FALSE.
* The remaining three return NULL.

In [ ]:

SELECT
true && true,
false && null,
null && false,
true && null,
null && true,
null && null
FROM x


The following cell shows a query where data is being [intepreted as NULL](https://druid.apache.org/docs/latest/misc/math-expr/#logical-operator-modes) according to non-SQL-compatible NULL-handling behavior.

In [ ]:
SELECT
100 && 11,
0.7 || 0.3,
100 && 0,
'troo' && 'true',
'troo' || 'true'
FROM X

## Scalar functions

In the following SQL, some simple string scalar functions are used to output a number of new values.

Run the cell to see how a NULL value affects results.

In [ ]:
sql='''
SELECT
  CONCAT("timezone-null",' timezone') AS "timezone",
  LENGTH("timezone-null") AS "length",
  REPLACE("timezone-null",'T',' timezone') AS "easyToRead",
  REVERSE("timezone-null") AS "backwards",
  COUNT(*) AS "events"
FROM "example-koalas-null"
GROUP BY 1, 2, 3, 4
HAVING COUNT(*) > 5000
'''

display.sql(sql)

## Aggregations

Run the following cell to see how COUNT operations on specific columns that contain NULL values are treated.

* A total number of rows is output as `totalRows`.
* A count of all rows with a NULL `timezone-null` is made and output as `nullRows`.
* A count is made of the number of rows where `timezone-null` contains a non-value, output as `nonNullRows`.
* The NULL and non-NULL row counts are added together, showing they total `totalRows`.

> When SQL-compatible NULL-handling is not on, rows containing either an empty string or a NULL are excluded as they are considered equivallent. Read more in the [NULL-handling](https://druid.apache.org/docs/latest/querying/sql-data-types/#null-values) documentation.

In [ ]:
sql='''
SELECT
  COUNT(*) AS "totalRows",
  COUNT(*) FILTER (WHERE "timezone-null" IS NULL) AS "nullRows",
  COUNT("timezone-null") AS "nonNullRows",
  COUNT(*) FILTER (WHERE "timezone-null" IS NULL) + COUNT("timezone-null") AS "totalRows-2"
FROM "example-koalas-null"
'''

display.sql(sql)

The next cell shows that NULL values are not included in COUNT DISTINCT operations.

In [ ]:
sql='''
SELECT
  COUNT(DISTINCT "timezone-null") AS "distinctValues",
  COUNT(DISTINCT "timezone-null") FILTER (WHERE "timezone-null" IS NOT NULL) AS "distinctNonNullValues",
  COUNT(DISTINCT "timezone-null") FILTER (WHERE "timezone-null" IS NULL) AS "distinctNullValues"
FROM "example-koalas-null"
'''

display.sql(sql)

The following query shows how NULLs are separated out during a GROUP BY operation.

In [ ]:
sql='''
SELECT
  "timezone-null",
  COUNT(*) AS "totalEvents",
  SUM("session_length") AS "totalSessionLength",
  STRING_FORMAT('%.3f',AVG("session_length")) AS "avgSessionLength",
  MAX("session_length") AS "maxSessionLength",
  MIN("session_length") AS "minSessionLength"
FROM "example-koalas-null"
GROUP BY 1
HAVING COUNT(*) > 1000
'''

display.sql(sql)

The `session_length-EDTonly` dimension only contains a session length in seconds when the timezone is EDT, otherwise it contains NULL.

Run the following SQL to see that NULLs are ignored when aggregations are calculated, giving a match to the EDT row from the results above.

* `totalEvents` is the complete number of rows in the data.
* `totalEventsEDT` has been added to show the number of events that are known to have been recorded in the EDT timezone.
* The remaining functions specifically reference the `session_length-EDTonly` dimension.

In [ ]:
sql='''
SELECT
  COUNT(*) AS "totalEvents",
  COUNT(*) FILTER (WHERE "timezone-null" = 'EDT') AS "totalEventsEDT",
  SUM("session_length-EDTonly") AS "totalSessionLength",
  STRING_FORMAT('%.3f',AVG("session_length-EDTonly")) AS "avgSessionLength",
  MAX("session_length-EDTonly") AS "maxSessionLength",
  MIN("session_length-EDTonly") AS "minSessionLength"
FROM "example-koalas-null"
'''

display.sql(sql)

In the following SQL are some aggregation functions that [return NULL](https://druid.apache.org/docs/latest/querying/sql-aggregations/) by default.

> The default returned value depends on whether SQL-compatible NULL-handling is on. This can lead to some unexpected behavior Read more in the [NULL-handling](https://druid.apache.org/docs/latest/querying/sql-data-types/#null-values) documentation on how to switch between modes.

In [ ]:
sql='''
SELECT
  SUM(something),
  MIN(),
  MAX(),
  AVG()
'''

display.sql(sql)

## Arrays

Use the dedicated [array functions](https://druid.apache.org/docs/latest/querying/sql-array-functions) to work with arrays.

In the following SQL, this function returns NULL if a value cannot be found in the ARRAY (out of range) or if it cannot be found.

> When SQL-compatible NULL-handling is not being used, the returned value would be -1. Read more [here](https://druid.apache.org/docs/latest/querying/sql-array-functions).

In [ ]:
SELECT

ARRAY_OFFSET(array, 5)
ARRAY_OFFSET_OF(array, thing)

Run the following cell to see [how UNNEST handles NULL values](https://druid.apache.org/docs/latest/querying/sql/#unnest).

Notice that a record corresponding to each NULL is returned, rather then being removing duplicates.

## Clean up

Run the following cell to remove the XXX used in this notebook from the database.

In [ ]:
druid.datasources.drop("example-koalas-null")

## Summary

* You learned this
* Remember this

## Learn more

* Read the documentation on:
  * Enabling and disabling [SQL-compatible NULL handling](https://druid.apache.org/docs/latest/querying/sql-data-types#null-values) using `druid.generic.useDefaultValueForNull`
  * How Druid stores [NULL during ingestion](https://druid.apache.org/docs/latest/design/segments#handling-null-values)
  * The default returned value for different [aggregations](https://druid.apache.org/docs/latest/querying/sql-aggregations/)
  * [Logical operator](https://druid.apache.org/docs/latest/misc/math-expr/#logical-operator-modes) modes
* If you tend to use native rather than SQL queries, read about the [NULL filter](https://druid.apache.org/docs/latest/querying/filters#null-filter) in the documentation.
* See the [table of default values](https://druid.apache.org/docs/latest/querying/sql-data-types/#standard-types) stored during ingestion when SQL-compatible NULL-handling is not turned on
* Follow the [notebook on GROUP BY](./01-groupby.ipynb) to see how NULL appears in [GROUPING SETS](https://druid.apache.org/docs/latest/querying/sql/#group-by)
* Try out other scalar functions with NULL - check out the dedicated notebooks on [datetime](./07-functions-datetime.ipynb), [string](./08-functions-strings.ipynb), and [IP address](./10-functions-ip.ipynb) functions for examples.

In [ ]:
# STANDARD CODE BLOCKS

# When just wanting to display some SQL results
display.sql(sql)

# When ingesting data:
display.run_task(sql)
sql_client.wait_until_ready('example-koalas-null')
display.table('example-koalas-null')

# When you want to make an EXPLAIN look pretty
print(json.dumps(json.loads(sql_client.explain_sql(sql)['PLAN']), indent=2))

# When you want a simple plot
df = pd.DataFrame(sql_client.sql(sql))
df.plot(x='x-axis', y='y-axis', marker='o')
plt.xticks(rotation=45, ha='right')
plt.gca().get_legend().remove()
plt.show()

# When you want to add some query context parameters
req = sql_client.sql_request(sql)
req.add_context("useApproximateTopN", "false")
resp = sql_client.sql_query(req)

# When you want to compare two different sets of results
df3 = df1.compare(df2, keep_equal=True)
df3